<a href="https://colab.research.google.com/github/superfreazy/ChatGPT/blob/main/SadTalker_Video_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 浪子之心科技
# 此代码仅为演示SadTalker驱动视频数字人


In [1]:
#@title 检测显卡
!nvidia-smi

Tue Feb 27 03:38:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title 一、下载源码并安装依赖
!git clone https://github.com/Zz-ww/SadTalker-Video-Lip-Sync

Cloning into 'SadTalker-Video-Lip-Sync'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 314 (delta 36), reused 32 (delta 32), pack-reused 260
Receiving objects: 100% (314/314), 4.56 MiB | 10.39 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
# 安装miniconda,安装虚拟环境，激活虚拟环境。
# 如果已经下载，就可以不用再执行!wget
%cd /content/SadTalker-Video-Lip-Sync
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
# 设置包
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')
# 安装python=3.8 并检查python版本
!conda install python=3.8 -y
!python --version
# 创建虚拟环境
!conda create --name SadTalker-Video-Lip-Sync python=3.8 -y
# 激活环境
!source activate SadTalker-Video-Lip-Sync
# 安装依赖
%cd /content/SadTalker-Video-Lip-Sync
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install ffmpeg
!python3.8 -m pip install -r requirements.txt
!pip install ninja
# 安装DAIN模型
!python -m pip install paddlepaddle-gpu==2.3.2.post112  \
-f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

/content/SadTalker-Video-Lip-Sync
--2024-02-27 03:38:38--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   211MB/s    in 0.4s    

2024-02-27 03:38:39 (211 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - c

In [41]:
#@title 二、下载模型和数据
# 已经从云盘复制了文件夹过来就不需要运行
%cd /content/SadTalker-Video-Lip-Sync//content/SadTalker-Video-Lip-Sync/checkpoints
!gdown https://drive.google.com/uc\?id\=1-07XiDSKZTGmqX3-WlNvfVAiCxE3fpMH
!rm -rf checkpoints
!unzip checkpoints.zip
!rm checkpoints.zip

/content/SadTalker-Video-Lip-Sync
Downloading...
From (original): https://drive.google.com/uc?id=1-07XiDSKZTGmqX3-WlNvfVAiCxE3fpMH
From (redirected): https://drive.google.com/uc?id=1-07XiDSKZTGmqX3-WlNvfVAiCxE3fpMH&confirm=t&uuid=fd5d0d24-8d53-4235-9f08-4b85d7dc32b6
To: /content/SadTalker-Video-Lip-Sync/checkpoints.zip
100% 3.82G/3.82G [00:40<00:00, 93.5MB/s]
Archive:  checkpoints.zip
   creating: checkpoints/
   creating: checkpoints/BFM_Fitting/
  inflating: checkpoints/BFM_Fitting/01_MorphableModel.mat  
  inflating: checkpoints/BFM_Fitting/BFM09_model_info.mat  
  inflating: checkpoints/BFM_Fitting/BFM_exp_idx.mat  
  inflating: checkpoints/BFM_Fitting/BFM_front_idx.mat  
  inflating: checkpoints/BFM_Fitting/Exp_Pca.bin  
  inflating: checkpoints/BFM_Fitting/facemodel_info.mat  
  inflating: checkpoints/BFM_Fitting/select_vertex_id.mat  
 extracting: checkpoints/BFM_Fitting/similarity_Lm3D_all.mat  
  inflating: checkpoints/BFM_Fitting/std_exp.txt  
   creating: checkpoints/__MACOS

In [34]:
#@title 三、上传视频
%cd /content/SadTalker-Video-Lip-Sync/examples/driven_video
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content/SadTalker-Video-Lip-Sync/examples/driven_video


KeyboardInterrupt: 

In [45]:
%cd /content/SadTalker-Video-Lip-Sync/checkpoints
!gdown https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth

/content/SadTalker-Video-Lip-Sync/checkpoints
Downloading...
From: https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
To: /content/SadTalker-Video-Lip-Sync/checkpoints/GFPGANv1.3.pth
100% 349M/349M [00:01<00:00, 233MB/s]


In [35]:
#@title 四、上传音频
%cd /content/SadTalker-Video-Lip-Sync/examples/driven_audio
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content/SadTalker-Video-Lip-Sync/examples/driven_audio


Saving tlp-zh.wav to tlp-zh.wav


In [26]:
%cd /content/SadTalker-Video-Lip-Sync/src/face3d
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content/SadTalker-Video-Lip-Sync/src/face3d


Saving extract_kp_videos.py to extract_kp_videos (1).py


In [46]:
#@title 五、进行推理
# 运行报错后，代码中将/content/SadTalker-Video-Lip-Sync/src/face3d/extract_kp_videos.py 第16行_2D 改为TWO_D
#!apt-get install libffi7
%cd /content/SadTalker-Video-Lip-Sync
!python inference.py --driven_audio '/content/SadTalker-Video-Lip-Sync/examples/driven_audio/tlp-zh.wav' \
            --source_video '/content/SadTalker-Video-Lip-Sync/examples/driven_video/me.mp4' \
            --enhancer 'lip' \
            --use_DAIN  \
              --time_step '0.5'

print('恭喜您高清数字人视频合并成功，结果在results中。')

/content/SadTalker-Video-Lip-Sync
./checkpoints/epoch_20.pth
./checkpoints/auido2pose_00140-model.pth
./checkpoints/auido2exp_00300-model.pth
./checkpoints/facevid2vid_00189-model.pth.tar
./checkpoints/mapping_00109-model.pth.tar
Traceback (most recent call last):
  File "inference.py", line 123, in <module>
    main(args)
  File "inference.py", line 60, in main
    enhancer_model = FaceEnhancement(base_dir='checkpoints', size=512, model='GPEN-BFR-512', use_sr=False,
  File "/content/SadTalker-Video-Lip-Sync/third_part/GPEN/gpen_face_enhancer.py", line 14, in __init__
    self.facedetector = RetinaFaceDetection(base_dir, device)
  File "third_part/GPEN/face_detect/retinaface_detection.py", line 27, in __init__
    self.load_model()
  File "third_part/GPEN/face_detect/retinaface_detection.py", line 51, in load_model
    pretrained_dict = torch.load(self.pretrained_path, map_location=torch.device('cpu'))
  File "/usr/local/lib/python3.8/site-packages/torch/serialization.py", line 699, in

In [22]:
#!strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep GLIBCXX
!rm /usr/local/lib/libstdc++.so.6
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /usr/local/lib

In [23]:
!strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep GLIBCXX

GLIBCXX_3.4
GLIBCXX_3.4.1
GLIBCXX_3.4.2
GLIBCXX_3.4.3
GLIBCXX_3.4.4
GLIBCXX_3.4.5
GLIBCXX_3.4.6
GLIBCXX_3.4.7
GLIBCXX_3.4.8
GLIBCXX_3.4.9
GLIBCXX_3.4.10
GLIBCXX_3.4.11
GLIBCXX_3.4.12
GLIBCXX_3.4.13
GLIBCXX_3.4.14
GLIBCXX_3.4.15
GLIBCXX_3.4.16
GLIBCXX_3.4.17
GLIBCXX_3.4.18
GLIBCXX_3.4.19
GLIBCXX_3.4.20
GLIBCXX_3.4.21
GLIBCXX_3.4.22
GLIBCXX_3.4.23
GLIBCXX_3.4.24
GLIBCXX_3.4.25
GLIBCXX_3.4.26
GLIBCXX_3.4.27
GLIBCXX_3.4.28
GLIBCXX_3.4.29
GLIBCXX_3.4.30
GLIBCXX_DEBUG_MESSAGE_LENGTH
